In [ ]:
!pip install scikit-learn==1.0
!pip install xgboost==1.4.2
!pip install catboost==0.26.1
!pip install pandas==1.3.3
!pip install radiant-mlhub==0.3.0
!pip install rasterio==1.2.8
!pip install numpy==1.21.2
!pip install pathlib==1.0.1
!pip install tqdm==4.62.3
!pip install joblib==1.0.1
!pip install matplotlib==3.4.3
!pip install Pillow==8.3.2
!pip install torch==1.9.1
!pip install plotly==5.3.1


In [ ]:
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('RuntimeWarning')

from radiant_mlhub import Collection
import tarfile
import os
from pathlib import Path
import json
from tqdm import tqdm

import datetime
import rasterio
import numpy as np
import pandas as pd
from collections import ChainMap
from collections import Counter
import gc
import os
from joblib import Parallel,delayed
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
import pandas as pd
import rasterio
import math
gc.collect()

9

In [ ]:
def get_date_format(month,day):
    '''
        Structures the dates in a particular format
    '''
    if (str(month)=='nan') or (str(day)=='nan'):
        return 'nan'
    else:
        if month>=10:
            if day>=10:
                return f'month_{str(int(month))}_day_{str(int(day))}'
            else:
                return f'month_{str(int(month))}_day_0{str(int(day))}'
        else:
            if day>=10:
                return f'month_0{str(int(month))}_day_{str(int(day))}'
            else:
                return f'month_0{str(int(month))}_day_0{str(int(day))}'

            
def attain_field_labels(tile_df):
    
    
    field_id_src             = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array           = field_id_src.read(1)
    
    
    return field_id_array

def get_bands(tile_df,date_time,band):
    try:
        source                   = rasterio.open(tile_df[(tile_df['dates']==date_time) & (tile_df['asset']==band)]['file_path'].values[0])
        band_array               = np.expand_dims(source.read(1).flatten(), axis=1)
        band_array               = np.reshape(band_array,(256,256))
        return band_array
        
    except:
        band_array               = np.array([np.nan]*65536)
        return band_array.reshape(256,256)


def get_aggregate_values(field_id_array):
    unique_fields          = np.unique(field_id_array)
    field_dictionary       = {}
    for fields in unique_fields:
        if fields!=0:
            
            field_assigned  = np.multiply(np.where(field_id_array==fields,1,0),field_id_array)
            
            field_assigned  = field_assigned.flatten()
            
            
            field2select    = np.unique(field_assigned)[1]
            field_distrib   = Counter(field_assigned)
            print('Field id ', fields, 'percentage ',(field_distrib[field2select])/65536)
            field_dictionary[fields] = (1000*(field_distrib[field2select]))/65536
            
    return field_dictionary       
            
        
    

In [ ]:
competition_test_df          = pd.read_csv('test_data_sentinel2.csv')
competition_test_df['month'] = pd.to_datetime(competition_test_df['datetime']).dt.month.values 
competition_test_df['day']   = pd.to_datetime(competition_test_df['datetime']).dt.day.values

competition_test_df['dates'] = competition_test_df.apply(lambda z: get_date_format(z['month'],z['day']),axis=1)

unique_dates                  = competition_test_df['dates'].unique()
unique_dates                  = np.array([z for z in unique_dates if 'nan' not in z])

print(f'Length of unique dates {len(unique_dates)}')

Length of unique dates 76


In [ ]:
date_dict                    = dict(zip(competition_test_df['datetime'].dropna().unique(),unique_dates))
date_dict                    = dict(sorted(date_dict.items(), key=lambda item: item[1]))
date_order_to_consider       = np.array(list(date_dict.values()))
tile_ids_test                = competition_test_df['tile_id'].unique()
date2parse                   = [key for key,_ in date_dict.items()]

In [ ]:
len(tile_ids_test)

1137

In [ ]:
def tile_id_fetch(tile_ids,date_specific,band,regular_update_field_dict):
    if tile_ids!=1951:
        
        tile_df                    = competition_test_df[competition_test_df['tile_id']==tile_ids]
        
        field_id_array = attain_field_labels(tile_df)
        
        field_agg      = get_aggregate_values(field_id_array)
        
        return field_agg
    else:
        return {}


In [ ]:


bands = ['B01']


for band in bands:
    overall_date_dictionary = {}
    for date_specific in tqdm(date_order_to_consider[:1]):

        regular_update_field_dict      = {}

        tile_field_aggregate           = Parallel(n_jobs=-1,timeout=100000,backend="multiprocessing", verbose=1)(delayed(tile_id_fetch)(tile_ids=i,date_specific=date_specific,band=band,regular_update_field_dict=regular_update_field_dict) for i in tile_ids_test)
        _                          = regular_update_field_dict.update(dict(ChainMap(*tile_field_aggregate)))
        print(len(regular_update_field_dict))       
               

        overall_date_dictionary['size_of_field'] = regular_update_field_dict
        
        
        gc.collect()

    df = pd.DataFrame(overall_date_dictionary).reset_index().rename({'index':'Field_id'},axis=1)
#     df.to_csv(f'/root/sentinel2_updated/Parallel_processing_train/Bands/mean_label.csv',index=False)



  0%|          | 0/1 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.


Field id  3699 percentage  0.00482177734375
Field id  73658 percentage  6.103515625e-05
Field id  1393 percentage  0.0119476318359375
Field id  12098 percentage  0.0115966796875
Field id  714 percentage  0.0016632080078125
Field id  103 percentage  0.0155029296875
Field id  401 percentage  0.0010223388671875
Field id  82528 percentage  0.0842132568359375
Field id  15025 percentage  0.001556396484375
Field id  9966 percentage  0.0005950927734375
Field id  12706 percentage  0.000885009765625
Field id  60 percentage  0.006805419921875
Field id  7920 percentage  0.003509521484375
Field id  57386 percentage  0.00048828125
Field id  1831 percentage  0.0002288818359375
Field id  1287 percentage  0.0010223388671875
Field id  116306 percentage  0.0427398681640625
Field id  32086 percentage  0.100921630859375
Field id  12538 percentage  0.04296875
Field id  7612 percentage  0.0026092529296875
Field id  25442 percentage  0.0082550048828125
Field id  13153 percentage  0.0032196044921875
Field id  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Field id  1420 percentage  0.0014190673828125
Field id  63712 percentage  0.000335693359375
Field id  49116 percentage  0.0085601806640625
Field id  92723 percentage  0.0267791748046875
Field id  116599 percentage  0.005218505859375
Field id  23865 percentage  0.0001068115234375
Field id  93956 percentage  0.0003204345703125
Field id  116617 percentage  0.0069580078125
Field id  22730 percentage  0.0415191650390625
Field id  38783 percentage  0.0001373291015625
Field id  59851 percentage  0.0034332275390625
Field id  42716 percentage  0.00372314453125
Field id  119132 percentage  0.0036468505859375
Field id  16507 percentage  0.0178680419921875
Field id  22367 percentage  0.0006561279296875
Field id  4559 percentage  0.00164794921875
Field id  31393 percentage  0.0016021728515625
Field id  90444 percentage  0.00341796875
Field id  5287 percentage  0.0118408203125
Field id  6396 percentage  0.0037994384765625
Field id  43874 percentage  0.0074005126953125
Field id  93796 percentage  0.0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

Field id  121037 percentage  0.0071258544921875
Field id  58718 percentage  0.006683349609375
Field id  120655 percentage  0.017578125
Field id  71375 percentage  0.007049560546875
Field id  38304 percentage  0.005615234375
Field id  28865 percentage  0.0194549560546875
Field id  117550 percentage  0.0049285888671875
Field id  43236 percentage  0.0284881591796875
Field id  30036 percentage  0.0035552978515625
Field id  57579 percentage  0.020843505859375
Field id  36776 percentage  0.001983642578125
Field id  34782 percentage  0.00482177734375
Field id  52803 percentage  0.001190185546875
Field id  78695 percentage  0.00885009765625
Field id  59104 percentage  0.009918212890625
Field id  18613 percentage  0.0054779052734375
Field id  61509 percentage  0.02813720703125
Field id  9249 percentage  0.0031585693359375
Field id  40405 percentage  0.09326171875
Field id  104373 percentage  4.57763671875e-05
Field id  6143 percentage  0.00872802734375
Field id  106579 percentage  0.00054931640

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Field id  101101 percentage  0.0124664306640625
Field id  101770 percentage  0.0002899169921875
Field id  102414 percentage  0.0034332275390625
Field id  104257 percentage  0.0020751953125
Field id  104506 percentage  0.0132293701171875
Field id  106082 percentage  0.000213623046875
Field id  106847 percentage  0.00067138671875
Field id  107384 percentage  0.0002899169921875
Field id  107406 percentage  0.0054168701171875
Field id  107665 percentage  0.003662109375
Field id  108370 percentage  0.00018310546875
Field id  108689 percentage  0.0004425048828125
Field id  109053 percentage  0.0037689208984375
Field id  109163 percentage  0.0002593994140625
Field id  109349 percentage  0.00042724609375
Field id  109456 percentage  0.002105712890625
Field id  110355 percentage  0.001434326171875
Field id  110508 percentage  0.0055999755859375
Field id  110543 percentage  0.0017242431640625
Field id  110879 percentage  0.0081329345703125
Field id  111002 percentage  0.002105712890625
Field id 

[Parallel(n_jobs=-1)]: Done 1137 out of 1137 | elapsed:   25.1s finished
100%|██████████| 1/1 [00:30<00:00, 30.91s/it]

35295


In [ ]:
df.head(10)

,Field_id,size_of_field
0,5,10.940552
1,10,31.631470
2,11,16.128540
3,17,11.596680
4,18,1.449585
5,27,11.230469
6,30,36.071777
7,31,5.157471
8,32,0.823975
9,39,1.693726


In [ ]:
df.to_csv(f'/root/sentinel2_updated/Test_data_prep/merged_test_data/size_of_field_test.csv',index=False)